In [4]:
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [6]:
model = "keepitreal/vietnamese-sbert"
embeddings = HuggingFaceEmbeddings(model_name = 'VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')

No sentence-transformers model found with name VoVanPhuc/sup-SimCSE-VietNamese-phobert-base. Creating a new one with mean pooling.


model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [11]:
loader = PyPDFLoader('Data/Rule.pdf')
document = loader.load()
splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50,
        )
chunks = splitter.split_documents(document)
vector_store = FAISS.from_documents(chunks, embedding=embeddings)
vector_store.save_local("vectordb")


Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 88 0 (offset 0)


In [12]:
vectorstore = FAISS.load_local("vectordb", embeddings, allow_dangerous_deserialization=True)

question = "Quy định về thời gian thực tập"
docs = vectorstore.similarity_search(question,k=3)
print(docs)

[Document(page_content='bắt đầu kỳ thực tập, thực tập sinh đăng ký lịch thực tập theo tháng với cán bộ phụ trách.  • Trường hợp nghỉ vào những buổi đã đăng ký thực tập cần báo trước cho cán bộ phụ trách kèm theo lý do cụ thể.  • Trường hợp thời gian thực tập không thể diễn ra theo kế hoạch đã đăng kỳ, với lý do hợp lý, thực tập sinh phải chủ động thông báo cho cán bộ phụ trách để xin phép thay đổi thời gian thực tập. • Mọi vắng mặt không lý do, hoặc không tuân thủ thời gian làm việc của công ty với lý do không hợp lý', metadata={'source': 'Data/Rule.pdf', 'page': 0}), Document(page_content='độ hòa đồng, tích cực học hỏi. • Thực tập sinh cần chủ động trong việc tìm hiểu, cố gắng nỗ lực thực hiện các vấn đề được giao. Tuân thủ Nội quy Công ty được xem như một tiêu chí để đánh giá kết quả thực tập của Thực tập sinh. Văn bản này có hiệu lực kể từ ngày ký  Hà Nội, ngày 01 tháng 05 năm 2014 Giám đốc               KOJI HOSAKA', metadata={'source': 'Data/Rule.pdf', 'page': 0}), Document(page_c

In [18]:
import os
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
model = genai.GenerativeModel('gemini-pro')

In [19]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def gennerate(question):
    retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 5})

    docs = retriever.get_relevant_documents(question)
    context = format_docs(docs)
    prompt = f"""
    Bạn là trợ lý cho các nhiệm vụ trả lời câu hỏi, hãy trả lời bằng tiếng Việt, lịch sự và thân thiện.
    Hãy trả lời câu hỏi dựa trên dữ liệu có trong đoạn ngữ cảnh
    Hãy trả lời không biết nêu không có thông tin trong đoạn ngữ cảnh

    Đoạn ngữ cảnh: {context}

    Câu hỏi: {question}
    """
    return model.generate_content(prompt).text

print(gennerate("Quy định về thời gian thực tập"))

- Thời gian làm việc của toàn công ty: từ 8: 30 đến 12h00 và từ 13:00 đến 17: 30 hàng ngày từ thứ Hai đến thứ Sáu.
- Thời gian thực tập tối thiểu của 1 thực tập sinh phần mềm là 20 giờ/ tuần.


In [23]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=700)
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        bot_message = gennerate(message)
        chat_history.append((message, bot_message))
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "d:\Hust Study\Intern\Training\.venv\Lib\site-packages\gradio\queueing.py", line 532, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Hust Study\Intern\Training\.venv\Lib\site-packages\gradio\route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Hust Study\Intern\Training\.venv\Lib\site-packages\gradio\blocks.py", line 1928, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Hust Study\Intern\Training\.venv\Lib\site-packages\gradio\blocks.py", line 1514, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Hust Study\Intern\Training\.venv\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_s